In [75]:
import pandas as pd
import numpy as np
import os

In [76]:
df=pd.read_csv("../../flight_data_raw.csv")

In [77]:
df.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,CANCELLATION_CODE,DIVERTED,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,V26
0,2018,4,26,4,UA,DTW,EWR,910,1107.0,117.0,...,NaN,0,73.0,488,33.0,0.0,3.0,0.0,84.0,NaN
1,2018,4,26,4,UA,EWR,RDU,1400,1349.0,-11.0,...,NaN,0,74.0,416,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,4,26,4,UA,SFO,EUG,1059,1115.0,16.0,...,NaN,0,61.0,451,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,4,26,4,UA,TPA,IAD,1858,1853.0,-5.0,...,NaN,0,108.0,811,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,4,26,4,UA,SFO,BOS,1630,1625.0,-5.0,...,NaN,0,316.0,2704,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
count=df.groupby(["ORIGIN","DEST"]).size()
count


ORIGIN  DEST
ABE     ATL      969
        CLT      955
        DTW      924
        FLL       73
        MDT        1
                ... 
YAK     CDV      361
        JNU      361
YNG     PIE        1
        SFB        1
YUM     PHX     1376
Length: 6365, dtype: int64

In [113]:
#this works to get airport list
#df2 = df[["ORIGIN","DEST"]].groupby('ORIGIN').aggregate(lambda tdf: tdf.unique().tolist())

{'ABE': {'ABE': nan,
  'ABI': nan,
  'ABQ': nan,
  'ABR': nan,
  'ABY': nan,
  'ACK': nan,
  'ACT': nan,
  'ACV': nan,
  'ACY': nan,
  'ADK': nan,
  'ADQ': nan,
  'AEX': nan,
  'AGS': nan,
  'AKN': nan,
  'ALB': nan,
  'ALO': nan,
  'AMA': nan,
  'ANC': nan,
  'APN': nan,
  'ART': nan,
  'ASE': nan,
  'ATL': 969.0,
  'ATW': nan,
  'AUS': nan,
  'AVL': nan,
  'AVP': nan,
  'AZA': nan,
  'AZO': nan,
  'BDL': nan,
  'BET': nan,
  'BFF': nan,
  'BFL': nan,
  'BGM': nan,
  'BGR': nan,
  'BHM': nan,
  'BIL': nan,
  'BIS': nan,
  'BJI': nan,
  'BKG': nan,
  'BLI': nan,
  'BLV': nan,
  'BMI': nan,
  'BNA': nan,
  'BOI': nan,
  'BOS': nan,
  'BPT': nan,
  'BQK': nan,
  'BQN': nan,
  'BRD': nan,
  'BRO': nan,
  'BRW': nan,
  'BTM': nan,
  'BTR': nan,
  'BTV': nan,
  'BUF': nan,
  'BUR': nan,
  'BWI': nan,
  'BZN': nan,
  'CAE': nan,
  'CAK': nan,
  'CDC': nan,
  'CDV': nan,
  'CGI': nan,
  'CHA': nan,
  'CHO': nan,
  'CHS': nan,
  'CID': nan,
  'CIU': nan,
  'CKB': nan,
  'CLE': nan,
  'CLL': na

In [115]:
len(df2.loc["ATL"]["DEST"])

166

Atlanta has 166 Airports they fly to!

Create the final output csv:

In [116]:
x = df.groupby(["ORIGIN","DEST"]).size()

idx = pd.MultiIndex.from_product((df.ORIGIN.unique(), df.DEST.unique()))

r = x.reindex(idx).dropna()

r.reset_index() \
          .groupby('level_0').apply(lambda x: dict(zip(x['level_1'],x[0])))\
          .to_csv("test.csv")
